In [44]:
# 1. Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# 2. Load training data
train = pd.read_csv('training_data.csv')
print("Shape:", train.shape)

Shape: (172848, 41)


/var/folders/nd/dj_lrx2d6j3fccjhjy5pqygc0000gn/T/ipykernel_81947/3146997977.py:9: DtypeWarning: Columns (31,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('training_data.csv')


In [45]:
train.head(3)

,general_sector,city,zip_code,specific_sector,state,contributor_type,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,...,governor_contribution_ratio,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio
0,Retired,MISSOULA,59802.0,Retired,MT,Individual,1.0,4,4,0,...,0.5,0.000000,0.0,1.0,0.5,0.5,0.0,0.0,0.0,0.0
1,Retired,CLEVELAND HTS,44118.0,Retired,OH,Individual,0.5,2,2,0,...,0.0,0.216202,0.5,1.0,0.5,0.0,0.0,0.0,0.0,0.0
2,Public Sector Unions,NEWARK,NaN,Teachers unions,NJ,Non-Individual,1.0,4,4,0,...,0.0,3.257082,0.0,1.0,1.0,0.0,0.0,-2.257082,0.0,0.0


In [46]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172848 entries, 0 to 172847
Data columns (total 41 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   general_sector                                 172848 non-null  object 
 1   city                                           171380 non-null  object 
 2   zip_code                                       168919 non-null  float64
 3   specific_sector                                172848 non-null  object 
 4   state                                          172848 non-null  object 
 5   contributor_type                               172848 non-null  object 
 6   winner_ratio                                   172848 non-null  float64
 7   candidacy_count                                172848 non-null  int64  
 8   candidacy_democratic_count                     172848 non-null  int64  
 9   candidacy_republican_count           

In [47]:
#unique datatypes:
unique_dtypes = train.dtypes.unique()
print("\nUnique data types in the dataset:")
print(unique_dtypes)    


Unique data types in the dataset:
[dtype('O') dtype('float64') dtype('int64')]


In [49]:
#columns that have more than one dtypes in:
from collections import Counter

# 1. Detect mixed‑type columns by mapping each value to its type
mixed_cols = []
for col in train.columns:
    # get set of types excluding NaNs
    types = set(type(v) for v in train[col].dropna())
    if len(types) > 1:
        mixed_cols.append((col, Counter(type(v) for v in train[col].dropna())))


for col, type_counts in mixed_cols:
    print(f"{col!r} has mixed types:")
    for t, cnt in type_counts.items():
        print(f"   {t.__name__}: {cnt}")


#change all those str representations of numbers in those columns change type into float64:
for col, type_counts in mixed_cols:
    train[col] = pd.to_numeric(train[col], errors='coerce').astype('float64')

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172848 entries, 0 to 172847
Data columns (total 41 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   general_sector                                 172848 non-null  object 
 1   city                                           171380 non-null  object 
 2   zip_code                                       168919 non-null  float64
 3   specific_sector                                172848 non-null  object 
 4   state                                          172848 non-null  object 
 5   contributor_type                               172848 non-null  object 
 6   winner_ratio                                   172848 non-null  float64
 7   candidacy_count                                172848 non-null  int64  
 8   candidacy_democratic_count                     172848 non-null  int64  
 9   candidacy_republican_count           

In [36]:
# list the sorted data entries in this column:
str_mask = train['governor_contribution_ratio'].apply(lambda x: isinstance(x, str))
string_series = train.loc[str_mask, 'governor_contribution_ratio']
print(string_series)



149673          #NAME?
152439    -0.000903944
156241     -0.03001959
158208    -0.121284903
158459    -0.148149131
159590    -0.112917482
162161     -7.50305499
163455    -0.046048185
Name: governor_contribution_ratio, dtype: object


In [37]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172848 entries, 0 to 172847
Data columns (total 41 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   general_sector                                 172848 non-null  object 
 1   city                                           171380 non-null  object 
 2   zip_code                                       168919 non-null  float64
 3   specific_sector                                172848 non-null  object 
 4   state                                          172848 non-null  object 
 5   contributor_type                               172848 non-null  object 
 6   winner_ratio                                   172848 non-null  float64
 7   candidacy_count                                172848 non-null  int64  
 8   candidacy_democratic_count                     172848 non-null  int64  
 9   candidacy_republican_count           

In [3]:
# 4. Summary statistics for numeric features
numeric_cols = train.select_dtypes(include='number').columns.tolist()
train[numeric_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
zip_code,168919.0,3.885463e+04,28782.805172,0.000000,19103.0000,30324.000000,63021.000000,99999.00
winner_ratio,172848.0,5.940265e-01,0.410736,0.000000,0.0000,0.666667,1.000000,1.00
candidacy_count,172848.0,6.055546e+00,25.560675,2.000000,2.0000,2.000000,4.000000,2026.00
candidacy_democratic_count,172848.0,3.270885e+00,14.016075,0.000000,0.0000,2.000000,3.000000,1284.00
candidacy_republican_count,172848.0,2.784661e+00,14.804989,0.000000,0.0000,1.000000,2.000000,1014.00
contribution_count,172848.0,9.664329e+00,92.532086,2.000000,2.0000,3.000000,7.000000,31652.00
contribution_democratic_count,172848.0,5.335486e+00,30.948342,0.000000,0.0000,2.000000,4.000000,3990.00
contribution_republican_count,172848.0,4.328844e+00,81.499628,0.000000,0.0000,1.000000,3.000000,31648.00
politician_challenger_count,172848.0,7.036066e-01,2.001822,0.000000,0.0000,0.000000,1.000000,140.00
politician_count,172848.0,5.037912e+00,15.138458,2.000000,2.0000,2.000000,4.000000,829.00


In [ ]:


# 5. Target distribution
plt.figure(figsize=(6,4))
sns.histplot(train['winner_ratio'], bins=30, kde=True)
plt.title('Distribution of winner_ratio')
plt.xlabel('winner_ratio (%)')
plt.ylabel('Count')
plt.show()

# 6. Univariate plots for key categorical features
cat_cols = ['general_sector','specific_sector','contributor_type','state']
for col in cat_cols:
    plt.figure(figsize=(8,3))
    sns.countplot(data=train, y=col, order=train[col].value_counts().index)
    plt.title(f'Counts by {col}')
    plt.tight_layout()
    plt.show()

# 7. Correlation heatmap (numeric block)
plt.figure(figsize=(12,10))
corr = train[numeric_cols].corr()
sns.heatmap(corr, cmap='coolwarm', center=0, fmt=".2f")
plt.title('Numeric feature correlations')
plt.show()

# 8. Load and preview network files
network_files = [
    'all_candidates_state_bipartite_weighted_network.csv',
    'federal_contributor_top100_contributors_network.csv',
    'state_contributor_top100_contributors_network.csv',
    'winning_candidates_state_bipartite_weighted_network.csv'
]

for fn in network_files:
    df_net = pd.read_csv(fn, index_col=0)
    print(f"\n--- {fn} ---")
    print("Shape:", df_net.shape)
    display(df_net.iloc[:5, :5])  # show a small block
    print("Value summary:")
    print(df_net.values.flatten()[::len(df_net)//5].round(2))  # sample some weights

# 9. (Optional) Build one bipartite graph and compute centrality
#    Here: all_candidates_state as example
df_bi = pd.read_csv('all_candidates_state_bipartite_weighted_network.csv', index_col=0)
G = nx.Graph()
# add candidate nodes (bipartite=0)
G.add_nodes_from(df_bi.index, bipartite=0)
# add contributor nodes (bipartite=1)
G.add_nodes_from(df_bi.columns, bipartite=1)
# add edges with weights
for cand in df_bi.index:
    for contrib, w in df_bi.loc[cand].items():
        if w > 0:
            G.add_edge(cand, contrib, weight=w)

# degree centrality for contributors
contrib_nodes = [n for n,d in G.nodes(data=True) if d['bipartite']==1]
deg_cent = nx.degree_centrality(G.subgraph(contrib_nodes))
# convert to DataFrame
deg_df = pd.DataFrame.from_dict(deg_cent, orient='index', columns=['degree_centrality'])
deg_df.sort_values('degree_centrality', ascending=False).head()